**수정해야 할 것** <br/>
해당 코드 옆에 `# 수정` 이라고 주석 달아놓겠습니다!
- `# 데이터 로드` 셀의 `critic_data`, `user_data`, `steam_review` read_csv
- `Doc2Vec` 셀의 model.save : 경로 수정

In [ ]:
# 패키지 설치

import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.models import load_model
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import nltk
import joblib
import pickle
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# 함수 모음

# 특수문자 제거
def remain_engnum(sentence):
  sentence = re.sub('[^a-zA-Z0-9]+', ' ', sentence)
  return sentence

# 희귀단어 제거
def remain_word(review):
  result = []
  for word in review:
    if word in total_tokenizer.word_index.keys():
      result.append(word)
  return result

# 불용어 제거
def remove_stopwords(text): 
    output= [i for i in text if i not in stop_words] 
    return output

# 추천 게임 출력
def gentleman_ver1(game1, game2, game3):
  d1 = pd.DataFrame(model.docvecs.most_similar([game1], topn=10), columns=['game', 'similarity1']) # game1 추천목록 & 유사도
  d2 = pd.DataFrame(model.docvecs.most_similar([game2], topn=10), columns=['game', 'similarity2']) # game2 추천목록 & 유사도
  d3 = pd.DataFrame(model.docvecs.most_similar([game3], topn=10), columns=['game', 'similarity3']) # game3 추천목록 & 유사도
  df = pd.merge(d1, d2, on='game', how='outer') # d1, d2 merge
  df = pd.merge(df, d3, on='game', how='outer') # d2, d3 merge

  for game in df['game']: # ['game'] 행 하나씩 돌면서 
    if game in [game1, game2, game3]: # game이 game1, game2,m game3에 해당하는 경우
      drop_index = df[df['game']==game].index
      df.drop(drop_index, axis='index', inplace=True) # 해당 행 drop

  # 추천 리스트 간 중복되는 게임이 없는 경우 각 추천리스트에서 2개씩 뽑음
  if len(df) == 30: 
    return [x[0] for x in model.docvecs.most_similar([game1], topn=2)] + [x[0] for x in model.docvecs.most_similar([game2], topn=2)] + [x[0] for x in model.docvecs.most_similar([game3], topn=2)]
  # 유사도 평균 내서 상위 6개 추출
  else : 
    df.fillna(0, inplace=True)
    df['mean'] = df.mean(axis=1, numeric_only=True)
    df.sort_values('mean', ascending=False, inplace=True)
    return list(df['game'])[:6]

In [ ]:
# 메타 전문가 리뷰

critic_data = pd.read_csv('./data/meta_final/metacrawl_meta_final.csv') # 수정

# game / review 열로 가져오기, 조건 설정
d1 = critic_data[(critic_data['platform']=='pc') & (critic_data['criticscore']>=75)][['title', 'criticcontent']]

# 이름 각각 game, score으로 변경
d1.rename(columns = {'title':'game', 'criticcontent':'review'}, inplace = True)
# 리뷰 NaN값인 행 제거
d1.dropna(subset=['review'], inplace=True) 

# 게임 이름에 전처리 : 특수문자 없애기, 띄어쓰기 없애기, 소문자 처리하기
d1['game'] = d1['game'].apply(lambda x:remain_engnum(x))
d1['game'] = d1['game'].str.replace(" ", "")
d1['game'] = d1['game'].str.lower()



# 메타 유저 리뷰

user_data = pd.read_csv("./data/meta_final/metacrawl_user_final.csv") # 수정

# game / review 열로 가져오기, 조건 설정
d2 = user_data[(user_data['platform']=='pc') & (user_data['userscore']>7.5)][['title', 'usercontent']]

# 이름 각각 user, game, score으로 변경
d2.rename(columns = {'title':'game', 'usercontent':'review'}, inplace = True)
# 리뷰 NaN값인 행 제거
d2.dropna(subset=['review'], inplace=True) 

# 게임 이름에 전처리 : 특수문자 없애기, 띄어쓰기 없애기, 소문자 처리하기
d2['game'] = d2['game'].apply(lambda x:remain_engnum(x))
d2['game'] = d2['game'].str.replace(" ", "")
d2['game'] = d2['game'].str.lower()



# 스팀 리뷰

steam_review = pd.read_csv("./data/steam_data/real_final.csv") # 수정

steam_review.dropna(subset=['review'], inplace=True) # 리뷰 NaN값인 행 제거
steam_review.dropna(subset=['weighted_vote_score'], inplace=True) # weighted_vote_score NaN값인 행 제거
steam_review.dropna(subset=['voted_up'], inplace=True) # voted_up NaN값인 행 제거
steam_review.dropna(subset=['title'], inplace=True) # title NaN값인 행 제거

d3 = steam_review[(steam_review['voted_up']==True) & (steam_review['weighted_vote_score']>=0.75)][['title', 'review']]

# 이름 각각 game, score으로 변경
d3.rename(columns = {'title':'game'}, inplace = True)

# 게임 이름에 전처리 : 특수문자 없애기, 띄어쓰기 없애기, 소문자 처리하기
d3['game'] = d3['game'].apply(lambda x:remain_engnum(x))
d3['game'] = d3['game'].str.replace(" ", "")
d3['game'] = d3['game'].str.lower()



# total_data  생성 : d1, d2, d3 합치기
total_data = pd.concat([d1, d2, d3])
# nan값 있는 행 제거
total_data.dropna(inplace=True)
# 게임 명 공백인 행 제거
drop_index = total_data[total_data['game']==''].index
total_data.drop(drop_index, axis='index', inplace=True)

In [ ]:
# 희귀단어 기준 계산
# tokenizer로 total_data['review'] 단어사전 만들고 희귀단어 비율 보기

tok = Tokenizer() # 토크나이저 할당
tok.fit_on_texts(total_data['review']) # 리뷰 데이터로 토크나이저 학습

# 등장 빈도수가 3회 이하 단어를 희귀단어로 취급
threshold = 3
total_cnt = len(tok.word_index) # 토크나이저가 생성한 단어사전의 단어 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합

for key, value in tok.word_counts.items():
    total_freq = total_freq + value
    if(value <= threshold):
        rare_cnt = rare_cnt + 1

# 희귀단어 기준
vocab_size = total_cnt- rare_cnt + 1 

In [ ]:
# 리뷰 전처리

# 영어, 숫자만 빼고 제거
total_data['review'] = total_data['review'].apply(lambda x:remain_engnum(x))

# 소문자화
total_data['review']= total_data['review'].apply(lambda x: x.lower())

# 게임 별로 리뷰 합치기
game_review = pd.DataFrame(total_data['game'].unique(), columns=['game'])

for game in total_data['game'].unique():
  game_review.loc[game_review['game']==game, 'review'] = " ".join(total_data[total_data['game']==game]['review'])

# 희귀단어 제거를 위한 토크나이저 생성
total_tokenizer = Tokenizer(num_words = vocab_size) # 희귀단어 제거한 단어사전 생성
total_tokenizer.fit_on_texts(game_review['review']) # 학습

# 단어사전에서 희귀단어 제거
words_frequency = [word for word, index in total_tokenizer.word_index.items() if index >= vocab_size + 1]
for word in words_frequency:
    del total_tokenizer.word_index[word] # 해당 단어에 대한 인덱스 정보를 삭제
    del total_tokenizer.word_counts[word] # 해당 단어에 대한 카운트 정보를 삭제

# 토큰화
game_review['review'] = game_review['review'].apply(lambda x:text_to_word_sequence(x))

# 희귀단어 제거 
game_review['review'] = game_review['review'].apply(lambda x:remain_word(x))

# 불용어 제거
stop_words = set(stopwords.words('english'))
game_review['review'] = game_review['review'].apply(lambda x:remove_stopwords(x))

In [ ]:
# Doc2Vec


# Doc2Vec에 들어가는 최종 데이터 TaggedDocument 생성

doc_df = game_review[['game','review']].values.tolist()
tagged_data = [TaggedDocument(words=_d, tags=[gid]) for gid, _d in doc_df]


# Doc2Vec 학습
max_epochs = 10

model = Doc2Vec(
    window=5, # 모델 학습 시 앞 뒤로 보는 단어 개수
    size=100, # 벡터차원 크기
    alpha=0.025, # learning rate
    min_alpha=0.025,# min learning rate. 학습 진행될수록 alpha값이 min_alpha로 근사
    min_count=3, # 학습에 사용할 최소 단어 빈도 수
    dm =1, # 학습 방법 : 1-pvdm, 0-pvdbow
    negative = 5, # complexity reudction. negative sampling 시 샘플링 할 단어 개수 지정
    seed = 9999)

model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save('./model/finaldatadoc2vec.model') # 수정

In [ ]:
# 저장된 모델 불러오기

model = Doc2Vec.load('./model/finaldatadoc2vec.model') # 수정